In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import torch

from gym_unity.envs import UnityEnv
from ddpg_agent import Agent

%matplotlib inline

In [ ]:
env_name = "E:/RL/Test2/Env_Rocket/Test"  # Name of the Unity environment binary to launch
env = UnityEnv(env_name, worker_id=0)
# Examine environment parameters
print(str(env))

In [ ]:
for episode in range(10):
    initial_observation = env.reset()
    episode_rewards = 0
    done = False
    while not done:
        observation, reward, done, info = env.step(env.action_space.sample())
        print(reward)
        print(done)
        episode_rewards += reward
#        if done:
#           text = 'Success' if reward>0 else 'Failed'
#            print(text, total_reward)
#            break
    print("Total reward this episode: {}".format(episode_rewards))

In [ ]:
agent = Agent(state_size=env.observation_space.shape[0], 
              action_size=env.action_space.shape[0], 
              random_seed=2)
#agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
#agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

In [ ]:
n_episodes = 1000
agent.noise.sigma = 2
scores = []
success = []
steps = 0
for i_episode in range(1, n_episodes+1):
    state = env.reset()
    score = 0
    while True:
        steps += 1
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            success += [reward>0]
            break
    scores += [score]
    print('\rEpisode', i_episode, 'Average Score: %.2f'%np.mean(scores[-20:]),
          'Success rate %.2f'%np.mean(success[-20:]),
          'Noise std: %.3f'%agent.noise.sigma,
          end="")
    agent.noise.sigma *= 0.995
    torch.save(agent.actor_local.state_dict(), 'easy_actor.pth')
    torch.save(agent.critic_local.state_dict(), 'easy_critic.pth')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(scores[-100:])
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
plt.plot(success[-100:])

In [ ]:
for _ in range(5):
    state = env.reset()
    total_reward = 0
    while True:
        action = agent.act(state, add_noise=False)
        state, reward, done = env.step(action)
        total_reward += reward
        if done:
            text = 'Success' if reward>0 else 'Failed'
            print(text, total_reward)
            break

In [ ]:
env.close()